# Programming Agent Memory

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

Letta agents persist information over time and restarts by saving data to a database. These lessons do not require past information. To enable a clean restart, the database is cleared before starting the lesson.

In [1]:
!rm  -f ~/.letta/sqlite.db

## Section 0: Setup a MemGPT client 

In [2]:
from helper import nb_print

In [3]:
from letta import create_client 

client = create_client() 

Saved Config:  /home/jovyan/.letta/config
📖 Letta configuration file updated!
🧠 model	-> gpt-4
🖥️  endpoint	-> http://jupyter-api-proxy.internal.dlai/rev-proxy/letta
Saved Config:  /home/jovyan/.letta/config
Saved Config:  /home/jovyan/.letta/config


In [4]:
from letta.schemas.llm_config import LLMConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 

## Section 1: Memory Blocks 

### Understanding ChatMemory

In [5]:
from letta.schemas.memory import ChatMemory  

In [6]:
chat_memory = ChatMemory(
    human="Name: Bob", 
    persona="You are a helpful assistant"
)

In [7]:
chat_memory.list_block_names()

['persona', 'human']

In [8]:
chat_memory.get_block("human")

Block(value='Name: Bob', limit=2000, name='human', template=False, label='human', description=None, metadata_={}, user_id=None, id='block-537ca87a-391c-40b8-9aca-fbf39638966c')

In [9]:
import inspect

In [10]:
print(inspect.getsource(chat_memory.core_memory_append))

    def core_memory_append(self: "Agent", name: str, content: str) -> Optional[str]:  # type: ignore
        """
        Append to the contents of core memory.

        Args:
            name (str): Section of the memory to be edited (persona or human).
            content (str): Content to write to the memory. All unicode (including emojis) are supported.

        Returns:
            Optional[str]: None is always returned as this function does not produce a response.
        """
        current_value = str(self.memory.get_block(name).value)
        new_value = current_value + "\n" + str(content)
        self.memory.update_block_value(name=name, value=new_value)
        return None



#### Context compilation 

In [11]:
chat_memory.get_prompt_template()

'{% for block in memory.values() %}<{{ block.name }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.name }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

In [12]:
chat_memory.compile()

'<persona characters="27/2000">\nYou are a helpful assistant\n</persona>\n<human characters="9/2000">\nName: Bob\n</human>'

## Section 2: Defining a custom memory module

### Defining a memory module


In [13]:
from letta.schemas.memory import ChatMemory
from letta.schemas.block import Block
from typing import Optional, List
import json

In [14]:
class TaskMemory(ChatMemory): 

    def __init__(self, human: str, persona: str, tasks: List[str]): 
        super().__init__(human=human, persona=persona, limit=2000) 
        self.link_block(
            name="tasks", 
            block=Block(
                limit=2000, 
                value=json.dumps(tasks), 
                name="tasks", 
                label="tasks"
            )
        )

    def task_queue_push(self: "Agent", task_description: str):
        """
        Push to a task queue stored in core memory. 

        Args:
            task_description (str): A description of the next task you must accomplish. 
            
        Returns:
            Optional[str]: None is always returned as this function 
            does not produce a response.
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        tasks.append(task_description)
        self.memory.update_block_value("tasks", json.dumps(tasks))
        return None

    def task_queue_pop(self: "Agent"):
        """
        Get the next task from the task queue 
 
        Returns:
            Optional[str]: The description of the task popped from the 
            queue, if there are still tasks in queue. Otherwise, returns
            None (the task queue is empty)
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        if len(tasks) == 0: 
            return None
        task = tasks[0]
        print("CURRENT TASKS: ", tasks)
        self.memory.update_block_value("tasks", json.dumps(tasks[1:]))
        return task

### Creating an agent with custom `TaskMemory`

In [15]:
task_agent_name = "task_agent"

task_agent_state = client.create_agent(
    name=task_agent_name, 
    system = open("task_queue_system_prompt.txt", "r").read(),
    memory=TaskMemory(
        human="My name is Sarah", 
        persona="You are an agent that must clear its tasks.", 
        tasks=[]
    )
)

In [16]:
message = "Add 'start calling me Charles'"  \
+ "and 'tell me a haiku about my name' as two seperate tasks."

In [17]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message=message
)
nb_print(response.messages)

CURRENT TASKS:  ['Start calling her Charles', 'Compose a haiku about Charles']
CURRENT TASKS:  ['Compose a haiku about Charles']


In [18]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
nb_print(response.messages)

In [19]:
client.get_core_memory(task_agent_state.id).get_block("tasks")

Block(value='[]', limit=2000, name='tasks', template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-ecc2ce46-b92c-4dd2-b39c-1b4cfe0e621f')

> copy the id='block-...' string, from the code cell above "client.get_core_memory...", and then paste into the code cell client.get_block('block-...')

In [21]:
client.get_block('cut_and_paste_id_from_above')

ValueError: Block does not exist